In [83]:
import csv
import datetime
import itertools
import numpy as np
import os.path
import pandas as pd
import pathlib

In [45]:
path = '/home/xandao/Documentos/resultados_gimp/identificacao_george/especie'

In [46]:
list_files = [file for file in pathlib.Path(path).rglob('mean.csv') if file.is_file()]
len(list_files)

75

In [47]:
list_extractor = {
    'lbp': [59],
    'surf64': [128, 256, 257],
    'surf128': [128, 256, 513],
    'mobilenetv2': [128, 256, 512, 1024, 1280],
    'resnet50v2': [128, 256, 512, 1024, 2048],
    'vgg16': [128, 256, 512]
}
index = [e + '_' + str(d) + '_' + m for e in list_extractor.keys() for d in reversed(list_extractor[e]) for m in ['mean', 'std']]

In [48]:
list_classifier = ['DecisionTreeClassifier', 'KNeighborsClassifier', 'MLPClassifier', 'RandomForestClassifier', 'SVC']
list_dim = [256, 400, 512]
list_segmented = ['manual', 'unet']
columns = [c + '_' + str(d) + '_' + s for c in list_classifier for s in sorted(list_segmented) for d in list_dim]

In [49]:
def create_sheet():
    return {
        'mean': pd.DataFrame(index=index, columns=columns),
        'time': pd.DataFrame(index=index, columns=columns),
        'folder': pd.DataFrame(index=index, columns=columns)
    }

sheet_rgb = create_sheet()
sheet_grayscale = create_sheet()
display(sheet_rgb['mean'])

,DecisionTreeClassifier_256_manual,DecisionTreeClassifier_400_manual,DecisionTreeClassifier_512_manual,DecisionTreeClassifier_256_unet,DecisionTreeClassifier_400_unet,DecisionTreeClassifier_512_unet,KNeighborsClassifier_256_manual,KNeighborsClassifier_400_manual,KNeighborsClassifier_512_manual,KNeighborsClassifier_256_unet,...,RandomForestClassifier_512_manual,RandomForestClassifier_256_unet,RandomForestClassifier_400_unet,RandomForestClassifier_512_unet,SVC_256_manual,SVC_400_manual,SVC_512_manual,SVC_256_unet,SVC_400_unet,SVC_512_unet
lbp_59_mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lbp_59_std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_257_mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_257_std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_256_mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_256_std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_128_mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf64_128_std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf128_513_mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
surf128_513_std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
ROUND_VALUE = 4

def round_mean(value):
     return '=ROUND(' + str(value) + ' * 100; ' + str(ROUND_VALUE) + ')'

def plus_minus_std(value):
     return '="±"&ROUND(' + str(value) + ' * 100; ' + str(ROUND_VALUE) + ')'

def get_classifier(path):
    classifier = list(filter(lambda x: x.lower() in str(path).lower(), list_classifier))

    if len(classifier) == 0:
        raise ValueError('classifier not available in list')

    return classifier[0]

def is_date(string):
    try:
        date = datetime.datetime.strptime(string, '%d-%m-%Y-%H-%M-%S')
        return True

    except ValueError:
        return False

def get_date(path):
    for p in str(path).split('/'):
        if is_date(p):
            return p
    return None


def set_values_sheet(column, date, index_folder, index_mean, index_std, mean, sheet, std):
    print(sheet.keys())
    sheet['mean'].loc[index_mean, column] = round_mean(mean)
    sheet['mean'].loc[index_std, column] = plus_minus_std(std)
    sheet['folder'].loc[index_folder, column] = date


def fill_sheet_mean_std(classifier, color_mode, date, filename, image_size, extractor, n_features, n_patch, segmented):
    sheet_mean = pd.read_csv(filename, sep=';', index_col=0, header=None)
    mean = sheet_mean.loc['mean_f1_sum'][1]
    std = sheet_mean.loc['std_f1_sum'][1]

    index_mean = extractor + '_' + n_features + '_' + 'mean'
    index_std = extractor + '_' + n_features + '_' + 'std'
    column = classifier + '_' + image_size + '_' + segmented

    if color_mode == 'grayscale':
        set_values_sheet(column, date, index_mean, index_mean, index_std, mean, sheet_grayscale, std)
    else:
        set_values_sheet(column, date, index_mean, index_mean, index_std, mean, sheet_rgb, std)

0.2339172677355828 0.0377736509004835
mobilenetv2_1024_mean mobilenetv2_1024_std DecisionTreeClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.2567374487500576 0.027380746583973
mobilenetv2_128_mean mobilenetv2_128_std DecisionTreeClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.2489946994271412 0.0292335267892332
mobilenetv2_1280_mean mobilenetv2_1280_std DecisionTreeClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.2389061650836714 0.0165296186636065
mobilenetv2_256_mean mobilenetv2_256_std DecisionTreeClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.2315587867340994 0.0190002606645272
mobilenetv2_512_mean mobilenetv2_512_std DecisionTreeClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.3342926681047731 0.0217965520346892
mobilenetv2_1024_mean mobilenetv2_1024_std KNeighborsClassifier_256_unet
dict_keys(['mean', 'time', 'folder'])
0.3635830728426247 0.0190126423536934
mobilenetv2_128_mean mobilenetv2_128_std KNeighborsClassifier_256

In [ ]:
def mean_top_k(classifier, color_mode, date, file, image_size, extractor, n_features, n_patch, segmented):
    f = str(file).replace('mean.csv', '')
    n_fold = 5

    list_top_k = []
    min = -1
    max = -1
    for i in range(0, n_fold):
        file_top_k = os.path.join(f, str(i), 'top_k', 'sum', 'top_k_sum.csv')

        if not os.path.isfile(file_top_k):
            raise FileNotFoundError(f'{file_top_k} not found')

        sheet_top_k = pd.read_csv(file_top_k, header=0, sep=';')
        min = sheet_top_k.loc[sheet_top_k['k'].idxmin()]['k']
        max = sheet_top_k.loc[sheet_top_k['k'].idxmax()]['k']
        list_top_k.append(sheet_top_k.to_dict('records'))

    if min == -1 and max == -1:
        raise ValueError('min or max not found')

    list_top_k = list(itertools.chain.from_iterable(list_top_k))

    list_each_k = []
    for i in range(min, max+1):
        values_k = [k['top_k_accuracy'] for k in list_top_k if k['k'] == i]
        list_each_k.append({'k': i, 'values': values_k, 'mean': np.mean(values_k)})

    display(pd.DataFrame(list_each_k))

In [ ]:
for file in sorted(list_files):
    sheet_info = pd.read_csv(str(file).replace('mean.csv', 'info.csv'), header=None, sep=';', index_col=0)

    classifier = get_classifier(file)
    color_mode = sheet_info.loc['color_mode'][1]
    image_size = sheet_info.loc['dim_image'][1]
    extractor = sheet_info.loc['extractor'][1]
    n_features = sheet_info.loc['data_n_features'][1]
    n_patch = sheet_info.loc['n_patch'][1]
    slice_patch = sheet_info.loc['slice'][1]
    segmented = 'unet' if 'unet' in str(file).lower() or 'u-net' in str(file).lower() else 'manual'
    date = get_date(file)

    fill_sheet_mean_std(classifier, color_mode, date, file, image_size, extractor, n_features, n_patch, segmented)
    mean_top_k(classifier, color_mode, date, file, image_size, extractor, n_features, n_patch, segmented)

In [51]:
sheet_grayscale['mean'].to_csv('grayscale.csv', sep=';', na_rep='', quoting=csv.QUOTE_ALL)
sheet_grayscale['mean'].to_excel('grayscale.xlsx', na_rep='', engine='xlsxwriter')
sheet_rgb['mean'].to_csv('rgb.csv', sep=';', na_rep='', quoting=csv.QUOTE_ALL)
sheet_rgb['mean'].to_excel('rgb.xlsx', na_rep='', engine='xlsxwriter')